## Assignment 6 - Image Classification

by:Yichin Tzou

### Convolutional Neural Network & Classification: 

The objective is to build an image classifier that is capable of properly identifying four different categories of image. 

The data consists of various train and test samples across the four categories of image. You will notice that the data for a specific category is a singular image that has been flipped, rotated, or slightly altered in some way. 

### Preparation 
1.First Install TensorFlow with Python's pip package manager

In [1]:
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

### 2.Setup/import

In [2]:
import tensorflow as tf

from tensorflow import keras

2023-05-09 16:51:25.079984: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import os, glob
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model


pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

## 1. Data Processing: 

The train & test data is pretty clean in terms of image data, but we will need to do a bit of prep work to use in our model. 

### a) Use the "ImageDataGenerator()" class from keras.processing.image to build out an instance called "train_datagen" with the following parameters: 

- rescale = 1./255
- shear_range = 0.2
- zoom_range = 0.2
- horizontal_flip = True

In [4]:
train_datagen=ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

### b) Then build your training set by using the method ".flow_from_directory()"

- path (where training data is stored)
- target_size = (64, 64)
- batch_size = 32
- class_mode = categorical 

In [5]:
train_generator = train_datagen.flow_from_directory(
        'dataset_train',  # this is the target directory
        target_size=(64, 64),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels


Found 88 images belonging to 4 classes.


### c) Take a look at your training set: 

What is the image shape of each training observation?

In [6]:
image_shape = train_generator.image_shape
image_shape

(64, 64, 3)

How many total classes do we need to predict on? 


In [7]:
total_classes = train_generator.num_classes
total_classes

4

## 2. Initial Classifier Build: 

Now use keras to build an initial image classifier with the following specifications.

Note: If you get lost, there is great documentation online and homework 7 included details on many of the layers used here.



In [8]:
# Create an instance of Sequential
classifier = Sequential()

# Add a Conv2D layer
classifier.add(Conv2D(32, kernel_size=(3, 3), input_shape=image_shape, activation='relu'))

# Add a MaxPooling2D layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Add another Conv2D layer
classifier.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

# Add another MaxPooling2D layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Add a Flatten layer
classifier.add(Flatten())

# Add a Dense layer
classifier.add(Dense(units=128, activation='relu'))

# Add a final Dense layer
classifier.add(Dense(units=total_classes, activation='softmax'))

# Compile the classifier
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## 3. Model Runs: 

This will be run various times with different numbers of steps_per_epoch and epochs. 

### a) Use .fit() with the training set. For the first run, use the following parameters: 

- steps_per_epoch = 3
- epochs = 3

In [9]:
classifier.fit(
    train_generator,
    steps_per_epoch=3,
    epochs=3
)

Epoch 1/3


2023-05-09 16:51:32.391401: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - 2s 230ms/step - loss: 1.7888 - accuracy: 0.2273
Epoch 2/3
3/3 [==============================] - 1s 245ms/step - loss: 1.0822 - accuracy: 0.5682
Epoch 3/3
3/3 [==============================] - 1s 272ms/step - loss: 0.7783 - accuracy: 0.7727


### b) save model to a file. 

In [10]:
classifier.save('my_model.h5')
print("Saved model")

Saved model


### c) Predict using the model built in step 2.

In [11]:

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')
print("Loaded model from disk")

# test data path
img_dir = "dataset_test" # Enter Directory of test set

# iterate over each test image
data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)

# print the files in the dataset_test folder 
for f in files:
    print(f)
    
# make a prediction and add to results 
data = []
results = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result = model.predict(img)
    r = np.argmax(result, axis=1)
    results.append(r)

results

Loaded model from disk
dataset_test/C033.png
dataset_test/1022.png
dataset_test/4011.png
dataset_test/1053.png
dataset_test/6051.png
dataset_test/4053.png
dataset_test/C014.png
dataset_test/6023.png
1/1 [==============================] - 0s 18ms/step


[array([3]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([2]),
 array([1]),
 array([1])]

### d) Determine accuracy.

Note: To determine accuracy, you will need to check the labels given to each class in the training data and manually label your test data. This will require you to look into the training data(images) in the dataset_train folder, and then determine how a category was coded in keras.

- Look into the training data(images) in the dataset_train folder, and then determine how a category was coded in keras 

In [12]:
# check category labels in training_set
train_generator.class_indices

{'category 1': 0, 'category 2': 1, 'category 3': 2, 'category 4': 3}

- look in the test data(images) in the dataset_test folder, and identify what category each images belongs to using images in the training set as references(there are only 8 test observations).

- Create a list to store the category/labels for the test data as the actual values.

In [13]:
test_label= [3, 0, 2, 0, 1, 2, 3, 1]

- Compare the predicted values to the actual values for the test set and calculate accuracy score

In [14]:
accuracy = accuracy_score(test_label, results)
accuracy

0.75

### e) Run this process for the following combinations:

* (steps_per_epoch: 1, epochs: 1)
* (steps_per_epoch: 1, epochs: 2)
* (steps_per_epoch: 1, epochs: 3)
* (steps_per_epoch: 2, epochs: 4)
* (steps_per_epoch: 2, epochs: 5)
* (steps_per_epoch: 2, epochs: 6)
* (steps_per_epoch: 3, epochs: 7)
* (steps_per_epoch: 3, epochs: 8)
* (steps_per_epoch: 5, epochs: 9)
* (steps_per_epoch: 5, epochs: 10)



In [15]:
steps_per_epoch = [1,1,1, 2,2,2, 3,3, 5,5]
epochs =list(np.arange(10)+1)

In [16]:
#run all the models from the above given combination
for i, j in list(zip(steps_per_epoch, epochs)):
    classifier = Sequential()
    classifier.add(Conv2D(filters = 32, kernel_size = (3,3), input_shape =  (64, 64,3),activation = "relu"))
    classifier.add(MaxPooling2D(pool_size = (2,2)))
    classifier.add(Conv2D(filters = 64, kernel_size = (3,3), activation = "relu"))
    classifier.add(MaxPooling2D(pool_size = (2,2)))
    classifier.add(Flatten())
    classifier.add(Dense(units = 128, activation = "relu"))
    classifier.add(Dense(units = 4, activation = "softmax"))
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    classifier.fit(train_generator,steps_per_epoch = i, epochs=j)
    name = "my_model_" + str(j) + ".h5"
    classifier.save(name)
    print(f"Saved model {name}")

2023-05-09 16:51:36.704328: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 1s 922ms/step - loss: 1.4550 - accuracy: 0.2500
Saved model my_model_1.h5
Epoch 1/2


2023-05-09 16:51:38.098731: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 1s 822ms/step - loss: 1.4073 - accuracy: 0.0417
Epoch 2/2
1/1 [==============================] - 0s 303ms/step - loss: 1.3811 - accuracy: 0.5625
Saved model my_model_2.h5
Epoch 1/3


2023-05-09 16:51:39.939638: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


...

3/3 [==============================] - 1s 230ms/step - loss: 2.3679 - accuracy: 0.2273
Epoch 2/8
3/3 [==============================] - 1s 234ms/step - loss: 1.0982 - accuracy: 0.5568
Epoch 3/8
3/3 [==============================] - 1s 228ms/step - loss: 0.7049 - accuracy: 0.7841
Epoch 4/8
3/3 [==============================] - 1s 265ms/step - loss: 0.5718 - accuracy: 0.8636
Epoch 5/8
3/3 [==============================] - 1s 252ms/step - loss: 0.3386 - accuracy: 0.9432
Epoch 6/8
3/3 [==============================] - 1s 218ms/step - loss: 0.2400 - accuracy: 0.9773
Epoch 7/8
3/3 [==============================] - 1s 222ms/step - loss: 0.1682 - accuracy: 0.9432
Epoch 8/8
3/3 [==============================] - 1s 224ms/step - loss: 0.2224 - accuracy: 0.9205
Saved model my_model_8.h5
Epoch 1/9


2023-05-09 16:52:09.871089: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


5/5 [==============================] - 2s 209ms/step - loss: 1.4836 - accuracy: 0.3295
Saved model my_model_9.h5
Epoch 1/10


2023-05-09 16:52:11.946747: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


5/5 [==============================] - 1s 119ms/step - loss: 2.4269 - accuracy: 0.3295
Saved model my_model_10.h5


In [17]:
accuracy_list = []
for j in (np.arange(10)+1):
    name = "my_model_" + str(j) + ".h5"
    # returns a compiled model
    # identical to the previous one
    model = load_model(name)
    print("Loaded model from disk")

    # make a prediction and add to results 
    data = []
    results = []
    for f1 in files:
        img = image.load_img(f1, target_size = (64, 64))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        data.append(img)
        result = model.predict(img)
        r = np.argmax(result, axis=1)
        results.append(r)
        
    score = accuracy_score(test_label,results)
    accuracy_list.append(score)

Loaded model from disk
1/1 [==============================] - 0s 17ms/step
Loaded model from disk
1/1 [==============================] - 0s 17ms/step
Loaded model from disk
1/1 [==============================] - 0s 17ms/step
Loaded model from disk
1/1 [==============================] - 0s 17ms/step
Loaded model from disk
1/1 [==============================] - 0s 17ms/step
Loaded model from disk
1/1 [==============================] - 0s 17ms/step
Loaded model from disk
1/1 [==============================] - 0s 19ms/step
Loaded model from disk
1/1 [==============================] - 0s 17ms/step
Loaded model from disk
1/1 [==============================] - 0s 17ms/step
Loaded model from disk
1/1 [==============================] - 0s 17ms/step


### f) Create a final dataframe that combines the accuracy across each combination.

In [18]:
df_accuracy = pd.DataFrame(list(zip(steps_per_epoch, epochs, accuracy_list)), columns=['Steps per Epoch', 'Epochs','Accuracy'])
df_accuracy

,Steps per Epoch,Epochs,Accuracy
0,1,1,0.250
1,1,2,0.750
2,1,3,0.625
3,2,4,0.875
4,2,5,0.875
5,2,6,0.875
6,3,7,0.750
7,3,8,0.875
8,5,9,0.625
9,5,10,0.250


## Conceptual Questions: 

## 4. Discuss the effect of the following on accuracy and loss (train & test): 

### - Increasing the steps_per_epoch

Increasing the steps_per_epoch means that more training samples will be processed within each epoch. This can potentially lead to better accuracy and reduce the loss during training, since the model can see and learn from more data during each epoch. However, 

### - Increasing the number of epochs

Increasing the number of epochs allows the model to see the entire training dataset multiple times. This can help improve accuracy and reduce the loss during training, by giving the model more opportunities to learn and refine its predictions. However, if the model starts to overfit the training data, increasing the number of epochs beyond a certain point can lead to a decrease in accuracy on unseen test data.



## 5. Name two uses of zero padding in CNN.
1. Keeping feature map size consistent: output feature map has the same dimensions as the input image
2. Reducing the border effect: helps maintaining consistent information across the entire input and output volumes, preventing information loss at the edges.

## 6. What is the use of a 1 x 1 kernel in CNN? 
The use of 1*1 kernal is to change the number of output feature maps

## 7. What are the advantages of a CNN over a fully connected DNN for this image classification problem?
1. Fewer parameters: A CNN has fewer parameters than a fully connected DNN, which makes it much faster to train, reduces the risk of overfitting, and requires much less training data.

2. Computational Efficiency: Due to parameter sharing and the localized nature of convolutions, CNNs are computationally more efficient compared to fully connected DNNs when dealing with high-dimensional input data such as images. CNNs leverage the convolutional operation, which reduces the number of parameters and the computational cost by reusing weights in different spatial locations.